# 🔄 Temporal Cycles

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import parmesan
from parmesan.units import units

Download and parse some history temperature data for Stuttgart from the [German Weather Service's Open Data Collection](https://opendata.dwd.de/climate_environment/CDC/):

In [ ]:
data = pd.read_csv(
    "https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/10_minutes/air_temperature/historical/10minutenwerte_TU_04926_20020621_20091231_hist.zip",
    delimiter=";",
    # nrows=50000,  # uncomment this to use less data
    parse_dates=["MESS_DATUM"],
    index_col="MESS_DATUM",
    na_values=[-999],
    infer_datetime_format=True,
)
data.tail()

## 📈 Plot the Timeseries

In [ ]:
fig, (axleft, axright) = plt.subplots(ncols=2, figsize=(15, 6))
data["TT_10"].plot(ax=axleft, legend=True)
axleft.set_title("Temperature in Stuttgart over the Years")
data["TT_10"].loc["2002-08-01":"2002-09-01"].plot(ax=axright, legend=True)
axright.set_title("Temperature in Stuttgart over one Month");

☝ There are clearly two dominant cycles here: a **seasonal** (yearly) cycle and a **diurnal** (daily) cycle.

## 📉 Plot a Spectrum 

There dominant cycles can be also seen in a spectrum:

In [ ]:
fig, ax = plt.subplots()  # new plot
data["TT_10"].interpolate().dropna().parmesan.spectrum().plot(
    ax=ax
)  # plot the spectrum
props = iter(
    tuple(plt.rcParams["axes.prop_cycle"])
)  # a changing iterator of matplotlib colors
# show typical periods as vertical lines
for interval in (
    1 * units("year"),
    1 * units("day"),
    12 * units("hours"),
    # in this example, the harmonics of 1 day are also clearly visible
    24 / 3 * units("hours"),
    24 / 4 * units("hours"),
    24 / 5 * units("hours"),
    24 / 6 * units("hours"),
    # ...
):
    ax.axvline(
        (1 / interval).to("Hz").m,
        linestyle="dashed",
        label=interval,
        zorder=-1,
        **next(props),
    )
ax.set_xscale("log")
ax.set_yscale("log")
ax.legend()

Note that the **seasonal** (yearly) cycle is the most prominent one (highest peak), followed by the **diurnal** (daily) cycle. 

The harmonics of the 1-day period (integer divisions of 24 hours) are also visible, but don't matter much here.

## 🌄🌇 Diurnal Cycle

Using the `temporal_cycle()`-[function](https://tue-umphy.gitlab.io/software/parmesan/api/parmesan.aggregate.html#parmesan.aggregate.temporal_cycle) we can aggregate the whole timeseries into time blocks and operate on them, e.g. build an average for every hour of the day:

In [ ]:
# aggregate every hour of a day
diurnal_cycle = data.parmesan.temporal_cycle(
    interval="D",  # We want to aggregate data for every Day (could also use "Y" for year)
    resolution="h",  # let's aggregate to every hour (could also use "m" for minute)
)
# Build the average over each hour of a day
mean_diurnal_cycle = diurnal_cycle.mean()
# Plot it
fig, ax = plt.subplots()
mean_diurnal_cycle["TT_10"].plot(
    ax=ax, legend=True, label="mean diurnal cycle"
)
ax.set_xlabel("hour of day")
ax.set_ylabel("Temperature [°C]");

## 🌱🌞🍃🌨 Seasonal Cycle

We can also plot the **seasonal** (yearly) cycle quickly in one line:

In [ ]:
data["TT_10"].parmesan.temporal_cycle(
    interval="Y", resolution="D"
).mean().plot()

☝ The x-Axis is the day in the year, the y-Axis the average temperature at that day across all years in the dataset.

## 🔎 Finer Resolution Control

With only `interval` and `resolution`, we are limited to `numpy.datetime`'s [units](https://numpy.org/doc/stable/reference/arrays.datetime.html#datetime-units) intervals (e.g. `D` for day, `h` for hour, etc...).

We can fine-tune that with the `modifier` argument. Suppose we want a seasonal cycle, but with a resolution of **10 days**, not one (e.g. to smooth out the curve):

In [ ]:
data["TT_10"].parmesan.temporal_cycle(
    interval="Y",  # we want a seasonal cycle (blocks of years)
    resolution="D",  # we want daily resolution
    # but we reduce that resolution by a factor of 10
    modifier=lambda days: (days / 10).astype(int),
).mean().plot()

☝ The x-Axis is the number of the 10-day interval in the year, the y-Axis the average temperature at that day across all years in the dataset.